In [ ]:
import os
import numpy as np
import MilneEddington as ME
import crisp
import time
import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero")
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero")
import inv_utils as iu
from helita.io import lp
from lp_scripts.get_fov_angle import fov_angle
from hmi_plot import plot_hmi_ic_mag, plot_sst_pointings

In [ ]:
import importlib
importlib.reload(iu)
print('reloaded') 

In [ ]:
### Small scale anemone jet 

# datadir = '/mn/stornext/d18/lapalma/reduc/2024/2024-05-21/CRISP/cubes_nb/'
# crisp_im = datadir + 'nb_6173_2024-05-21T10:19:04_10:19:04=0-52_stokes_corrected_im.fits'
# blos_cube = datadir + 'Blos.6173_2024-05-21T10:19:04.icube'
# bhor_cube = datadir + 'Bhor.6173_2024-05-21T10:19:04.icube'
# xorg = 1110
# xsize = 800
# yorg = 570
# ysize = 800
# tt = 8
# is_north_up = True
# shape = 'circle'
# scale = 0.044 # arcsec/pixel
# crop = False

In [ ]:
### Sunspot data set close to center also observed with Hinode

# datadir = '/mn/stornext/d18/lapalma/reduc/2020/2020-08-07/CRISP/cubes_nb/'
# crisp_im = datadir + 'nb_6173_2020-08-07T08:22:14_scans=0-56_stokes_corrected_im.fits'
# blos_cube = datadir + 'Blos.6173_2020-08-07T08:22:14.icube'
# bhor_cube = datadir + 'Bhor.6173_2020-08-07T08:22:14.icube'
# xorg = 200
# xsize = 480
# yorg = 360
# ysize = 400
# tt = 0
# scale = 0.058 # arcsec/pixel
# is_north_up = False
# crop = False
# shape = 'square'

In [ ]:
### QS dataset used by Aditi

# datadir = '/mn/stornext/d18/lapalma/reduc/2021/2021-06-22/CRISP/cubes_nb/'
# crisp_im = datadir + 'nb_6173_2021-06-22T08:17:48_scans=0-162_stokes_corrected_im.fits'
# blos_cube = datadir + 'Blos.6173_2021-06-22T08:17:48.icube'
# bhor_cube = datadir + 'Bhor.6173_2021-06-22T08:17:48.icube'
# xsize = 256
# ysize = 256
# xorg = 273
# yorg = 420
# tt = 41
# scale = 0.058 # arcsec/pixel
# shape = 'square'
# crop = False
# is_north_up = False

In [ ]:
# Load the configuration from the JSON file
config = iu.load_config('input_config.json')
# Check the input configuration
config = iu.check_input_config(config, pprint=True, confirm=True)

In [ ]:

# Extract the parameters
data_dir = config['data_dir']
save_dir = config['save_dir']
crisp_im = config['crisp_im']
xorg = config['xorg']
xsize = config['xsize']
yorg = config['yorg']
ysize = config['ysize']
tt = config['time_index']
scale = config['scale']
is_north_up = config['is_north_up']
crop = config['crop']
shape = config['shape']
best_frame = config['best_frame']
contrasts = config['contrasts']
hmi_con_series = config['hmi_con_series']
hmi_mag_series = config['hmi_mag_series']
email = config['email']

In [ ]:

xrange = [xorg, xorg + xsize]
yrange = [yorg, yorg + ysize]

In [ ]:
fits_info = iu.get_fits_info(crisp_im)
fits_header = iu.load_fits_header(crisp_im)
t_obs = fits_info['avg_time_obs']
fov = fov_angle(t_obs)
print(f'FOV angle: {fov:.2f} deg')

In [ ]:

iu.plot_contrast(contrasts, figsize=(8, 4))
iu.plot_image(best_frame, title=f'Frame: {tt}', cmap='gray', scale=scale)


In [ ]:
nx = fits_info['nx']
ny = fits_info['ny']
mu = fits_info['mu']
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
tobs = fits_info['all_start_times'][tt]
tstart = fits_info['start_time_obs']
hplnt = fits_info['hplnt']
hpltt = fits_info['hpltt']
print(f'x1 = {x1}\nx2 = {x2}\ny1 = {y1}\ny2 = {y2}\ntobs = {tobs}')

In [ ]:
x_list = np.linspace(x1, x2, num=nx)
y_list = np.linspace(y1, y2, num=ny)
if crop:
    x_list = x_list[xrange[0]:xrange[1]]
    y_list = y_list[yrange[0]:yrange[1]]
    x1 = x_list[0]
    x2 = x_list[-1]
    y1 = y_list[0]
    y2 = y_list[-1]
    nx = xsize
    ny = ysize
print(f'x1: {x1:.2f}, x2: {x2:.2f}, y1: {y1:.2f}, y2: {y2:.2f}')
print(f'nx: {nx}, ny: {ny}')

In [ ]:
if crop:
    draw_rectangle = False
    draw_circle = False
    buffer = 5
else:
    if shape == 'circle':
        draw_rectangle = False
        draw_circle = True
        buffer = 0
    else:
        draw_rectangle = True
        draw_circle = False
        buffer = 5
if is_north_up:
    rot_fov = 0
else:
    rot_fov = fov
    buffer = 15


In [ ]:
plot_sst_pointings(tstart, hmi_con_series, hplnt, hpltt,figsize=(8, 8), email=email, save_dir=save_dir)

In [ ]:
plot_hmi_ic_mag(tobs, hmi_con_series, hmi_mag_series, email, x1, x2, y1, y2, draw_rectangle=draw_rectangle, height=56, width=56, rot_fov=rot_fov, save_dir=save_dir, draw_circle=draw_circle, radius=87, figsize=(10, 5), overwrite=False, buffer=buffer, enhance_ic=False, enhance_m=False)

In [ ]:
iu.plot_crisp_image(crisp_im, ss=0, ww=0, figsize=(8,8), fontsize=8, rot_fov=fov, north_up=not(is_north_up), crop=crop, xrange=xrange, yrange=yrange, xtick_range=[x1,x2], ytick_range=[y1,y2])

In [ ]:
#
# Decide to work in float32 or float64
#
dtype = 'float32'
nthreads = 96
#
# Load data, wavelength array and cmap
#
class container:
    def __init__(self):
        pass
l = container()
container.iwav = iu.get_wavelengths(crisp_im)
container.d = iu.load_crisp_fits(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
container.cmap = iu.load_crisp_cmap(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 
mask = iu.get_nan_mask(crisp_im, tt=tt,crop=crop, xrange=xrange, yrange=yrange) 
print(mask.shape)
iu.plot_crisp_image(crisp_im, tt=tt, crop=crop, xrange=xrange, yrange=yrange) 

In [ ]:

# Minimum step:
dw = np.min(np.diff(l.iwav))
# dw = round((lambda*10. - lc) * 1000.) ; offset in mA
dw = round(dw*1000.)/1000. # avoid floating point errors

# ==============================================================================

# The inversions need to account for the instrumental
# profile, which involve convolutions. The convolutions
# must be done in a wavelength grid that is at least
# 1/2 of the FWHM of the instrumental profile. In the
# case of CRISP that would be ~55 mA / 2 = ~27.5 mA
#
# Get finer grid for convolutions purposes
# Since we only observed at the lines, let's create
# two regions, one for each line
#
# The observed line positions are not equidistant, the
# Fe I 6301 points only fit into a regular grid of 5 mA
# whereas the Fe I 6302 can fit into a 15 mA grid
#
iw, idx = iu.find_grid(l.iwav, dw)  # Fe I 6173

# ==============================================================================


#
# Now we need to create a data cube with the fine grid
# dimensions. All observed points will contribute to the
# inversion. The non-observed ones will have zero weight
# but will be used internally to properly perform the
# convolution of the synthetic spectra
#


ny, nx = l.d.shape[0:2]
obs = np.zeros((ny, nx, 4, iw.size), dtype=dtype, order='c')

for ss in range(4):
    for ii in range(idx.size):
        obs[:, :, ss, idx[ii]] = l.d[:, :, ss, ii]

# ==============================================================================
#
# Create sigma array with the estimate of the noise for
# each Stokes parameter at all wavelengths. The extra
# non-observed points will have a very large noise (1.e34)
# (zero weight) compared to the observed ones (3.e-3)
# Since the amplitudes of Stokes Q,U and V are very small
# they have a low imprint in Chi2. We can artificially
# give them more weight by lowering the noise estimate.
#
sig = np.zeros((4, iw.size), dtype=dtype) + 1.e32
sig[:, idx] = 5.e-3
sig[1:3, idx] /= 9.0
sig[3, idx] /= 4.0

# ==============================================================================
#
# Init Me class. We need to create two regions with the
# wavelength arrays defined above and a instrumental profile
# for each region in with the same wavelength step
#
tw = (np.arange(iw.size, dtype=dtype)-iw.size//2)*dw

# ==============================================================================
# Central wavelength of the line:
l0 = iw[iw.size//2]
tr = crisp.crisp(l0).dual_fpi(tw, erh=-0.001)

regions = [[iw, tr/tr.sum()]]
lines = [int(l0)]
me = ME.MilneEddington(regions, lines, nthreads=nthreads, precision=dtype)

In [ ]:

#
# Init model parameters
#
iPar = np.float64([1500, 2.2, 1.0, -0.5, 0.035, 50., 0.1, 0.24, 0.7]) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
Imodel = me.repeat_model(iPar, ny, nx)


In [ ]:
#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations) of simple pixel-wise inversion
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=6, nIter=100, chi2_thres=1, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)


In [ ]:
masked_chi2_mean = iu.masked_mean(chi2, mask)
print(f'Masked chi2 mean: {masked_chi2_mean}')
if  masked_chi2_mean < 20:
    size_filter = 11
elif masked_chi2_mean < 50:
    size_filter = 21
else:
    size_filter = 31
Imodel = iu.parallel_median_filter(Imodel, size_filter=21)
iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
#
# Run second cycle
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(Imodel, obs, sig, nRandom=2, nIter=200, chi2_thres=1, mu=mu)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))
iu.plot_inversion_output(Imodel,mask,scale=scale, save_fig=False)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:

#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations)
#
t0 = time.time()
alphas=np.float32([2, 0.5, 2, 0.01, 0.1, 0.01, 0.1, 0.01, 0.01])
mo, syn, chi2 = me.invert_spatially_regularized(Imodel, obs, sig,  nIter=200, chi2_thres=1, mu=mu, alpha=30., alphas=alphas, method=1, delay_bracket=3)
t1 = time.time()
print("dT = {0}s -> <Chi2> (including regularization) = {1}".format(t1-t0, chi2))

#
# Correct velocities for cavity error map from CRISP
#
mos = np.squeeze(mo) # Remove the singleton dimension in the model and make the shape (ny, nx, 9) from (1, ny, nx, 9)
mos[:,:,3] += (l.cmap * 10) / l0 * 2.9e5
# mos[:,:,3] += l.cmap+0.45 # The 0.45 is a global offset that seems to make the umbra at rest

In [ ]:
iu.plot_inversion_output(mos,mask,scale=scale, save_fig=False)
iu.plot_mag(mos,mask,scale=scale, save_fig=False)

In [ ]:
iu.plot_sst_blos_bhor(blos_cube, bhor_cube, tt=tt,xrange=xrange, yrange=yrange, figsize=(20,10), fontsize=12, crop=crop)

In [ ]:
errors = me.estimate_uncertainties(mos, obs, sig, mu=mu)

In [ ]:
importlib.reload(iu)

In [ ]:
from einops import rearrange
mos_im = rearrange(mos, 'ny nx nparams -> nparams ny nx')
errors_im = rearrange(errors, 'ny nx nparams -> nparams ny nx')

In [ ]:
for i in range(9):
    iu.masked_stats(mos[:,:,i], mask)

In [ ]:
inversion_mask_replacements = [0, 0, 0, 0, 0, 0, 0, 0, 0] # Blos, inc, azi, v_los, v_dop, line op, damping, s0, s1

In [ ]:
masked_mos = np.zeros_like(mos)
for i in range(9):
    masked_mos[:,:,i] = iu.masked_data(mos[:,:,i], mask, replace_val=inversion_mask_replacements[i])


In [ ]:
iu.plot_inversion_output(masked_mos,scale=scale, save_fig=False)

In [ ]:
masked_errors = np.zeros_like(errors)
for i in range(9):
    masked_errors[:,:,i] = iu.masked_data(errors[:,:,i], mask, replace_val=inversion_mask_replacements[i], fix_inf=True)
iu.plot_inversion_output(masked_errors,scale=scale, save_fig=False)

In [ ]:
for i in range(9):
    iu.masked_stats(errors[:,:,i], mask)

In [ ]:
b_err = iu.masked_data(errors[:,:,0], mask)
print(np.nanmean(b_err))
print(np.nanmin(b_err))
print(np.nanmax(b_err))

In [ ]:
importlib.reload(iu)

In [ ]:
importlib.reload(iu)
iu.plot_image(masked_errors[:,:,1], scale=scale, title='B_tot (G)', save_fig=False, clip=True, vmax=1, vmin=0)

In [ ]:
# apply masked_data to all components or errors and save as masked_errors
masked_errors = np.zeros((ny, nx, 9), dtype=dtype)
for i in range(9):
    masked_errors[:,:,i] = iu.masked_data(errors[:,:,i], mask)

In [ ]:
importlib.reload(iu)
minc = iu.masked_data(errors[:,:,1], mask, replace_val=0)
print(np.min(minc))
print(np.max(minc))
print(np.median(minc))

In [ ]:
iu.plot_inversion_output(masked_errors, mask, scale=scale, save_fig=False)

In [ ]:
iu.plot_output(mos,mask,scale=scale)
iu.plot_mag(Imodel,mask,scale=scale, save_fig=False)

In [ ]:
importlib.reload(iu)

In [ ]:
## save the results as fits files with the same header as the input data
iu.save_fits(mos, fits_header, 'temp/inv_mos.fits', overwrite=True)

In [ ]:
ff = iu.load_fits_data('temp/inv_mos.fits')

In [ ]:
hh = iu.load_fits_header('temp/inv_mos.fits')

#### Things to complete
- [ ] Move all the inputs to a dictionary and later save them in the header of the output file. Also add the best seeing frame number.
- [ ] Move the preprocessing steps like plotting and FOV details as an optional but default true step
- [ ] Save fits with [blos, theta, phi, vlos + errors + mask] for each frame (temporarily) and later combine for final fits
- [ ] Check for option to convert to fcube and icube formats using ispy or helita tools
- [ ] Add option to do only one frame separately if user wants.
- [ ] Add fov angle and other inputs needed for ambiguity resolution and remap in header

#### To do for final cube
- [ ] Pick the best seeing frame from the dataset
- [ ] Run the full inversion for the best seeing frame
- [ ] Use this output as an initial guess for the other frames
